In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import pygal
import matplotlib.pyplot as plt
import cufflinks as cf
from sklearn.preprocessing import KBinsDiscretizer
import unicodedata
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.utils import resample
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif,f_regression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge, Lars,  LinearRegression
from sklearn.metrics import roc_auc_score,r2_score,confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
cf.go_offline()
pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',300)

In [ ]:
df=pd.read_pickle("Tabla_Proyecto_Limpia_forma1.pkl")

In [ ]:
quitar=['TMC','Start_Time', 'End_Time', 'Description', 'Street','Side','City','County','Zipcode',
          'Airport_Code','Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
       'Wind_Direction', 'Wind_Speed(mph)', 'Weather_Condition','City_Freq','Weather', 'Temperature(F)_Escalado', 'Humidity(%)_Escalado',
       'Pressure(in)_Escalado', 'Visibility(mi)_Escalado',
       'Wind_Speed(mph)_Escalado','Day']

In [ ]:
columnas=[x for x in df.columns if x not in quitar]

In [ ]:
df=df[columnas]

In [ ]:
df.info()

In [ ]:
df["Amenity"]=df["Amenity"]*1
df["Crossing"]=df["Crossing"]*1
df["Junction"]=df["Junction"]*1
df["Station"]=df["Station"]*1
df["Stop"]=df["Stop"]*1
df["Traffic_Signal"]=df["Traffic_Signal"]*1
df["Hour"]=df["Hour"].astype(str)
df["Year"]=df["Year"].astype(str)
df["Side_moda"]=(df["Side_moda"]=="R")*1
df["Weekend"]=df["Weekday"].map(lambda x: 1 if x=="Sat" or x=="Sun" else 0)

In [ ]:
df["Source"].value_counts()

In [ ]:
df1=df[df["Source"]=="MapQuest"].copy()
df1=df1.reset_index(drop=True)

## definimos variables

In [ ]:
cat=["State","Timezone","Hour","Weekend",
     "Month","Side_moda","Wind_Direction_moda","Weather_moda"]
cont=[x for x in df.columns if x not in cat+["Severity","Source","Distance(mi)","Weekday",
                                            "Year","TimeMinutes"]]

In [ ]:
#df["Severity"].value_counts(1)
df1["Severity"].value_counts(1)

In [ ]:
dic={1:0,2:0,3:1,4:1}
#df["target"]=df["Severity"].replace(dic)
df1["target"]=df1["Severity"].replace(dic)

# Continuamos

In [ ]:
X = df1[cont].join(pd.get_dummies(df1[cat]))
y = df1["target"]

In [ ]:
print('\nUnique values :  \n',X.nunique())

# k-best

In [ ]:
kb = SelectKBest(k=30, score_func=f_classif)

In [ ]:
kb.fit(X, y)

In [ ]:
ls_best= [x for x, y in zip(X.columns, kb.get_support()) if y]

In [ ]:
ls_best

In [ ]:
X = X[ls_best]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Remuestreo

In [ ]:
X_train.shape

In [ ]:
X_train=X_train.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)

In [ ]:
X_train["target"]=y_train

In [ ]:
X_train.shape

In [ ]:
X_train["target"].value_counts()

In [ ]:
df_majority = X_train[X_train.target==0].copy()
df_minority = X_train[X_train.target==1].copy()

In [ ]:
df_majority=df_majority.reset_index(drop=True)
df_minority=df_minority.reset_index(drop=True)

In [ ]:
df_sampled = resample(df_minority,#df_majority 
                                 replace=True,     #False
                                 n_samples=1010756,    # to match class
                                 random_state=123)

In [ ]:
#df = pd.concat([df_minority, df_sampled],axis=0)
X_train = pd.concat([df_majority, df_sampled],axis=0)
y_train=X_train["target"]

In [ ]:
X_train=X_train.drop(['target'], axis=1)

In [ ]:
y_train.value_counts()

In [ ]:
del df,df_majority,df_minority,df_sampled

### Modelado

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
logreg.score(X_test, y_test)

In [ ]:
ls_scores = cross_val_score(X=X_train, y=y_train, cv = 4, estimator=logreg, scoring="roc_auc")

In [ ]:
np.mean(ls_scores), np.std(ls_scores)

In [ ]:
roc_auc_score(y_score=logreg.predict(X_test), y_true=y_test)

In [ ]:
confusion_matrix(y_true=y_test, y_pred=logreg.predict(X_test))

In [ ]:
pd.DataFrame(list(zip(ls_best,logreg.coef_[0]))).sort_values(by = 1)

In [ ]:
param_grid = {"penalty" : ['l1', 'l2', 'elasticnet'],
    "tol": [0.00001, 0.0000001, 0.01],
    "solver" : [ 'liblinear', 'saga'],
     "C":[x/100 for x in range(0, 101,5)]         
}

In [ ]:
np.prod(list(map(len, param_grid.values())))

In [ ]:
%%time
clf = RandomizedSearchCV(n_iter=50, estimator=logreg, param_distributions=param_grid, cv=4, error_score=-1000, n_jobs=-1, scoring="roc_auc", verbose=5)
clf.fit(X_train, y_train)
print("Best score: " + str(clf.best_score_))

In [ ]:
clf.best_estimator_.get_params

In [ ]:
LogisticRegression?

In [ ]:
roc_auc_score(y_score=clf.best_estimator_.predict(X_test), y_true=y_test)

In [ ]:
confusion_matrix(y_true=y_test, y_pred=clf.best_estimator_.predict(X_test))

In [ ]:
pd.DataFrame(list(zip(ls_best,clf.best_estimator_.coef_[0]))).sort_values(by = 1)

In [ ]:
summary = pd.DataFrame(clf.cv_results_)

In [ ]:
summary.sort_values(by = "rank_test_score")

In [ ]:
#pd.to_pickle(clf.best_estimator_, "regresion_logistica.pickle")

In [ ]:
modelo_ant=pd.read_pickle("regresion_logistica.pickle")### ROC 0.7271650753733115

In [ ]:
roc_auc_score(y_score=modelo_ant.predict(X_test), y_true=y_test)

In [ ]:
columnas_mejor_modelo=['Start_Lat', 'Amenity', 'Crossing','Junction', 'Station',
 'Stop', 'Traffic_Signal', 'State_Freq', 'Airport_Freq', 'Weekend', 'Side_moda',
 'State_AZ', 'State_CA', 'State_CT', 'State_GA', 'State_IL', 'State_LA', 'State_MN',
 'State_MO', 'State_NC', 'State_OK', 'State_RI', 'State_SC', 'State_TX','State_VA',
 'Timezone_US/Central', 'Timezone_US/Pacific', 'Hour_7', 'Hour_8', 'Weather_moda_Fair']

#### Modelo preservado

In [ ]:
modelo=pd.read_pickle("regresion_logistica.pickle")

In [ ]:
pd.DataFrame(list(zip(ls_best,modelo.coef_[0]))).sort_values(by = 1)

In [ ]:
roc_auc_score(y_score=modelo.predict(X_test), y_true=y_test)

In [ ]:
confusion_matrix(y_true=y_test, y_pred=modelo.predict(X_test))

In [ ]:
pd.DataFrame(list(zip(ls_best,modelo.coef_[0]))).sort_values(by = 1)

# Modelo continuo

### volver a generar el df

In [ ]:
df=pd.read_pickle("Tabla_Proyecto_Limpia_forma1.pkl")

In [ ]:
quitar=['TMC','Start_Time', 'End_Time', 'Description', 'Street','Side','City','County','Zipcode',
          'Airport_Code','Temperature(F)', 'Humidity(%)', 'Pressure(in)'
        , 'Visibility(mi)', 'Wind_Speed(mph)',
       'Wind_Direction', 'Weather_Condition','City_Freq','Weather', 'Temperature(F)_Escalado', 'Humidity(%)_Escalado',
       'Pressure(in)_Escalado', 'Visibility(mi)_Escalado',
       'Wind_Speed(mph)_Escalado','Day']

In [ ]:
columnas=[x for x in df.columns if x not in quitar]

In [ ]:
df=df[columnas]

In [ ]:
df["Amenity"]=df["Amenity"]*1
df["Crossing"]=df["Crossing"]*1
df["Junction"]=df["Junction"]*1
df["Station"]=df["Station"]*1
df["Stop"]=df["Stop"]*1
df["Traffic_Signal"]=df["Traffic_Signal"]*1
df["Hour"]=df["Hour"].astype(str)
df["Year"]=df["Year"].astype(str)
df["Side_moda"]=(df["Side_moda"]=="R")*1
df["Weekend"]=df["Weekday"].map(lambda x: 1 if x=="Sat" or x=="Sun" else 0)

In [ ]:
df=df[df["Source"]=="MapQuest"].copy()
df=df.drop(columns=["Source"])
df=df.reset_index(drop=True)

In [ ]:
df=df[df["TimeMinutes"]<=df["TimeMinutes"].quantile(.99)].copy()
df=df.reset_index(drop=True)

In [ ]:
sns.boxplot(x="Severity", y="TimeMinutes",showfliers=False ,data=df)#Distance(mi)

In [ ]:
df.describe(np.arange(.05,1,.05))#[.99]

In [ ]:
df.columns

In [ ]:
pol=[ 'Start_Lat', 'Start_Lng','Temperature(F)_Escalado_KNN',
 'Humidity(%)_Escalado_KNN',
 'Pressure(in)_Escalado_KNN',
 'Visibility(mi)_Escalado_KNN',
 'Wind_Speed(mph)_Escalado_KNN']

In [ ]:
df.shape

In [ ]:
df1=df.sample(frac=.6,random_state=1)

In [ ]:
#del df
df1.shape

In [ ]:
#cortes=["Temperature(F)","Humidity(%)","Pressure(in)","Visibility(mi)","Wind_Speed(mph)"]

In [ ]:
#df1[cortes].describe()

In [ ]:
#for var in cortes:
#    df[f"cut_{var}"]=pd.cut(df[var],bins=7).astype(str)

In [ ]:
cat=["State","Timezone","Hour","Weekend",
     "Month","Side_moda","Wind_Direction_moda","Weather_moda"]#,"cut_Temperature(F)","cut_Humidity(%)","cut_Pressure(in)","cut_Visibility(mi)","cut_Wind_Speed(mph)"
cont=[x for x in df1.columns if x not in cat+["Severity","Source","Distance(mi)","Weekday",
                                            "Year","TimeMinutes"]]#+cortes

In [ ]:
cont

### Definimos variables

In [ ]:
poly = PolynomialFeatures(3)
df_pol = poly.fit_transform(df[pol])
df_pol = pd.DataFrame(columns=poly.get_feature_names(), data=df_pol)
df=pd.concat([df,df_pol],axis=1)

In [ ]:
cont2=[x for x in df.columns if x not in cat+["Severity","Source","Distance(mi)","Weekday",
                                            "Year","TimeMinutes"]]

In [ ]:
X = df1[cont].join(pd.get_dummies(df1[cat]))
y = df1["TimeMinutes"]#Distance(mi)

In [ ]:
del df1

In [ ]:
y.describe()

#### k-best

In [ ]:
kb = SelectKBest(k=50, score_func=f_classif)

In [ ]:
kb.fit(X, y)

In [ ]:
ls_best= [x for x, y in zip(X.columns, kb.get_support()) if y]

In [ ]:
ls_best

In [ ]:
X=X[ls_best]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Modelos sin optimizar

######  Regresion_lineal

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
r2_score(y_pred=model.predict(X_test), y_true=y_test)

In [ ]:
ls_medias = cross_val_score(estimator=model, X=X_test, y = y_test, cv = 4, n_jobs=-1, scoring="r2")
np.mean(ls_medias), np.std(ls_medias)

######  Regresion_Ridge

In [ ]:
model = Ridge()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
r2_score(y_pred=model.predict(X_test), y_true=y_test)

In [ ]:
ls_medias = cross_val_score(estimator=model, X=X_test, y = y_test, cv = 4, n_jobs=-1, scoring="r2")
np.mean(ls_medias), np.std(ls_medias)

######  Regresion_Lasso

In [ ]:
model = Lasso()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
ls_medias = cross_val_score(estimator=model, X=X_test, y = y_test, cv = 4, n_jobs=-1, scoring="r2")
np.mean(ls_medias), np.std(ls_medias)

######  Bayesian_Ridge

In [ ]:
model = BayesianRidge()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
ls_medias = cross_val_score(estimator=model, X=X_test, y = y_test, cv = 4, n_jobs=-1, scoring="r2")
np.mean(ls_medias), np.std(ls_medias)

######  Regresion_Lars

In [ ]:
model = Lars()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
ls_medias = cross_val_score(estimator=model, X=X_test, y = y_test, cv = 4, n_jobs=-1, scoring="r2")
np.mean(ls_medias), np.std(ls_medias)

######  Regresion_ElasticNet

In [ ]:
model = ElasticNet()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
ls_medias = cross_val_score(estimator=model, X=X_test, y = y_test, cv = 4, n_jobs=-1, scoring="r2")
np.mean(ls_medias), np.std(ls_medias)

#### Descartamos Lasso y ElasticNet

# Optimizamos Parámetros

In [ ]:
### La regresión Lineal no la podemos optimizar
dc_scores = {}

In [ ]:
model = Ridge()

In [ ]:
Ridge?

In [ ]:
param_grid = {
    "alpha": [x for x in range(1, 100)] + [y/10 for y in range(10)],
    "tol": [0.00001, 0.0000001, 0.01],
    "solver": ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

In [ ]:
np.prod(list(map(len, param_grid.values())))

In [ ]:
%%time
clf = RandomizedSearchCV(n_iter=25, estimator=model, param_distributions=param_grid, cv=4, error_score=-1000, n_jobs=-1, scoring="r2", verbose=5)
clf.fit(X_train, y_train)

In [ ]:
dc_scores[str(model).split("(")[0]] = {"model": clf.best_estimator_, "score": clf.best_score_}

In [ ]:
dc_scores

In [ ]:
clf.best_estimator_.get_params#'score': 0.20996117265901562

In [ ]:
clf.best_estimator_.score(X_test, y_test)

In [ ]:
#pd.to_pickle(clf.best_estimator_, "regresion_Ridge.pickle")

In [ ]:
columnas=pd.DataFrame()
columnas["Continuas"]=X_test.columns

In [ ]:
#columnas.to_csv("Columnas_modelo_continuo_timeminutes.csv")

In [ ]:
clf.best_estimator_.coef_

In [ ]:
clf.best_estimator_.score(X, y)### subio a 0.21007397052815446

# Modelo a partir de variables de Texto- descartado pero tiene cosas útiles

In [ ]:
def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().split())
    return cleaned_text
###############################################
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w,"v") for w in w_tokenizer.tokenize(text) if not w.isdigit()]
def token_join(tokens):
    s=" "
    return s.join(tokens)

In [ ]:
df=pd.read_csv("US_Accidents_June20.csv")

In [ ]:
df=df[df["Source"]=="MapQuest"][["Severity","Description"]].sample(frac=.7,random_state=1)

In [ ]:
df=df.dropna()
df=df.reset_index(drop=True)

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [ ]:
df["Description_clean"]=df["Description"].apply(clean_text)
df["Description_clean2"]=df["Description_clean"].apply(lemmatize_text)
df["Description_clean2"]=df["Description_clean2"].apply(token_join)

In [ ]:
df["len_text_raw"] = df["Description"].str.len()
df["n_dots"] = df["Description"].str.split(".").str.len() - 1
df["n_lower_ratio_len"] = df["Description"].map(lambda x:sum(map(str.islower, x))) / df["len_text_raw"]
df["n_upper_ratio_len"] = df["Description"].map(lambda x:sum(map(str.isupper, x))) / df["len_text_raw"]
df["n_words"] = df["Description"].str.split(" ").str.len()
df["n_letters"] = df["Description"].map(lambda x:sum(map(str.isalpha, x)))
df["n_lower_ratio_letters"] = df["Description"].map(lambda x:sum(map(str.islower, x))) / df["n_letters"]
df["n_upper_ratio_letters"] = df["Description"].map(lambda x:sum(map(str.isupper, x))) / df["n_letters"]
df["n_digit_ratio_letters"] = df["Description"].map(lambda x:sum(map(str.isdigit, x))) / df["n_letters"]

In [ ]:
stop_words=stopwords.words('english')

In [ ]:
df.head()

In [ ]:
#cv = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 1), min_df=1, max_features=100)
cv=CountVectorizer(stop_words=stop_words,ngram_range=(1, 1),min_df=8,max_features=70)

In [ ]:
cv.fit(df["Description_clean2"])

In [ ]:
ls_features = ["len_text_raw", "n_dots", "n_lower_ratio_len", "n_upper_ratio_len", "n_words", "n_letters", "n_lower_ratio_letters", "n_upper_ratio_letters"]

In [ ]:
X = df[ls_features].fillna(0).join(pd.DataFrame(data = cv.transform(df[["Description_clean2"]]).todense(), columns = cv.get_feature_names()))
y = df["Severity"]

In [ ]:
#del X,y
X.fillna(0,inplace=True)

In [ ]:
X.describe()

In [ ]:
kb=SelectKBest(k=8,score_func=f_classif)

In [ ]:
kb.fit(X,y)

In [ ]:
ls_best=[x for x,y in  zip(X.columns,kb.get_support()) if y]

In [ ]:
X=X[ls_best]
#del df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train.ravel())

In [ ]:
nb.score(X_train, y_train.ravel())

In [ ]:
ls_scores = cross_val_score(X=X_train, y=y_train, cv=4, n_jobs=-1, estimator=nb, scoring="accuracy")

In [ ]:
np.mean(ls_scores), np.std(ls_scores)

In [ ]:
confusion_matrix(y_true=y_test, y_pred=nb.predict(X_test))

In [ ]:
nb.coef_[0]

In [ ]:
nb.score

# Crédit Scoring

In [ ]:
def IV(df, var, tgt):
    aux = df[[var, tgt]].groupby(var).agg(["count", "sum"])
    aux["evento"] = aux[tgt, "sum"]
    aux["no_evento"] = aux[tgt, "count"] - aux[tgt, "sum"]
    aux["%evento"] = aux["evento"] / aux["evento"].sum()
    aux["%no_evento"] = aux["no_evento"] / aux["no_evento"].sum()
    aux["WOE"] = np.log(aux["%no_evento"] / aux["%evento"])
    aux["IV"] = (aux["%no_evento"] - aux["%evento"])*aux["WOE"]
    return aux["IV"].sum()
###############################################
def WOE(df, var, tgt):
    aux = df[[var, tgt]].groupby(var).agg(["count", "sum"])
    aux["evento"] = aux[tgt, "sum"]
    aux["no_evento"] = aux[tgt, "count"] - aux[tgt, "sum"]
    aux["%evento"] = aux["evento"] / aux["evento"].sum()
    aux["%no_evento"] = aux["no_evento"] / aux["no_evento"].sum()
    aux["WOE"] = np.log(aux["%no_evento"] / aux["%evento"])
    aux.columns = aux.columns.droplevel(1)
    aux = aux[["WOE"]].reset_index().rename(columns={"WOE": f"W_{var}"})
    df = df.merge(aux, on = var, how = "left")
    return df

In [ ]:
df=pd.read_pickle("Tabla_Proyecto_Limpia_forma1.pkl")

In [ ]:
df.columns

In [ ]:
quitar=['TMC','Start_Time', 'End_Time', 'Description', 'Street','Side','City','County','Zipcode',
          'Airport_Code','Temperature(F)_Escalado_KNN', 'Humidity(%)_Escalado_KNN', 'Pressure(in)_Escalado_KNN', 'Visibility(mi)_Escalado_KNN',
       'Wind_Direction', 'Wind_Speed(mph)_Escalado_KNN', 'Weather_Condition','City_Freq','Weather', 'Temperature(F)_Escalado', 'Humidity(%)_Escalado',
       'Pressure(in)_Escalado', 'Visibility(mi)_Escalado',
       'Wind_Speed(mph)_Escalado','Day']

In [ ]:
columnas=[x for x in df.columns if x not in quitar+['Year','County_Freq','State_Freq','Airport_Freq','TimeMinutes','Distance(mi)']]

In [ ]:
df=df[columnas]

In [ ]:
df["Amenity"]=df["Amenity"]*1
df["Crossing"]=df["Crossing"]*1
df["Junction"]=df["Junction"]*1
df["Station"]=df["Station"]*1
df["Stop"]=df["Stop"]*1
df["Traffic_Signal"]=df["Traffic_Signal"]*1
df["Hour"]=df["Hour"].astype(str)
df["Side_moda"]=(df["Side_moda"]=="R")*1

In [ ]:
df1=df[df["Source"]=="MapQuest"].copy()
df1=df1.reset_index(drop=True)

In [ ]:
del df
df1=df1.drop(columns=["Source"])
df1["Severity"]=df1["Severity"].replace({1:0,2:0,3:1,4:1})

In [ ]:
df1.head()

In [ ]:
tgt=["Severity"]
cont=["Start_Lat","Start_Lng","Temperature(F)","Humidity(%)","Pressure(in)","Visibility(mi)",
     "Wind_Speed(mph)",]
cat=["State","Timezone","Hour","Weekday","Month","Wind_Direction_moda","Weather_moda"]
dicotomicas=[x for x in df1.columns if x not in tgt+cont+cat]

In [ ]:
dicotomicas

In [ ]:
df1.shape

In [ ]:
df1=df1.dropna()
df1=df1.reset_index(drop=True)

In [ ]:
df1.shape

In [ ]:
2019139/2028102

In [ ]:
#treeDisc=DecisionTreeDiscretiser(cv=3,scoring='roc_auc',random_state=777)# regresa probabilidades
discretizer = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')

In [ ]:
X=discretizer.fit_transform(df1[cont],df1[tgt])

In [ ]:
X=pd.DataFrame(X,columns=cont)

In [ ]:
X=X.join(df1[dicotomicas+cat+tgt])

In [ ]:
for col in X.columns:
    display(X[col].value_counts(True).reset_index())

In [ ]:
new_disc=['Start_Lat', 'Start_Lng', 'Temperature(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Amenity',
       'Crossing', 'Junction', 'Station', 'Stop', 'Traffic_Signal',
       'Side_moda', 'State', 'Timezone', 'Hour', 'Weekday', 'Month',
       'Wind_Direction_moda', 'Weather_moda']

In [ ]:
df_iv = pd.DataFrame(columns=["iv"])
for var in new_disc:
    df_iv.loc[var, "iv"] = IV(df = X, var = var, tgt = "Severity")

In [ ]:
df_iv.sort_values(by = "iv", ascending = False)

In [ ]:
ls_best = df_iv[df_iv["iv"] != np.inf].sort_values(by = "iv", ascending=False).head(13).index

In [ ]:
for var in ls_best:
    X = WOE(df = X, var = var, tgt = "Severity")

In [ ]:
ls_woe = [x for x in X.columns if x.startswith("W_")]

In [ ]:
ls_woe

In [ ]:
del df1
Z=X[ls_woe]
y=X[tgt]

In [ ]:
Z.isnull().sum()

## Traintestsplit

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Z, y,random_state=777)

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(X_train, y_train.values.ravel())

In [ ]:
logreg.score(X_train, y_train)

In [ ]:
ls_scores = cross_val_score(X=X_train, y=y_train, cv = 4, estimator=logreg, scoring="roc_auc")

In [ ]:
np.mean(ls_scores), np.std(ls_scores)

In [ ]:
roc_auc_score(y_score=logreg.predict(X_test), y_true=y_test)## score 0.6786631923530955

In [ ]:
variables_usadas_en_los_modelos=['Side_moda', 'Traffic_Signal', 'State', 'Crossing', 'Weekday',
       'Junction', 'Stop', 'Hour', 'Timezone', 'Amenity', 'Station',
       'Start_Lng', 'Weather_moda']

In [ ]:
modelo=LogisticRegression()

In [ ]:
param_grid = {"penalty" : ['l1', 'l2', 'elasticnet'],
    "tol": [0.00001, 0.0000001,0.0000000001, 0.01],
    "solver" : [ 'liblinear', 'saga'],
     "C":[x/100 for x in range(0, 101,5)] }

In [ ]:
%%time
clf = RandomizedSearchCV(n_iter=20, estimator=modelo, param_distributions=param_grid, cv=4, error_score=-1000, n_jobs=-1, scoring="roc_auc", verbose=5)
clf.fit(X_train, y_train)
print("Best score: " + str(clf.best_score_))

In [ ]:
clf.best_estimator_.get_params

In [ ]:
roc_auc_score(y_score=clf.best_estimator_.predict(X_test), y_true=y_test)#0.6786394394134154

In [ ]:
#pd.to_pickle(clf.best_estimator_,"credit_scoring.pkl")
cs=pd.read_pickle("credit_scoring.pkl")

In [ ]:
pdo = 8
base_score = 140
base_odds = 100
factor = pdo/np.log(2)
offset = base_score - (factor*np.log(base_odds))
m = len(ls_woe)

In [ ]:
alpha = cs.intercept_
betas = cs.coef_[0]

In [ ]:
del Z,y,X_train,X_test,y_train,y_test

In [ ]:
for feat, beta in zip(ls_woe, betas):
    X["P_" + feat[2:]] = X[feat].apply(lambda x:((-beta*x + (alpha/m))*factor) + (offset/m)).astype(int)

In [ ]:
X["score"] = X[[x for x in X.columns if x.startswith("P_")]].sum(axis = 1)

In [ ]:
test=X[["score","Severity"]].groupby(by="score").agg("count")

In [ ]:
test.iplot(kind='bar',title='Frecuencia en Scores',color=['red'],
                            theme='white',yTitle='Frecuencia',xTitle='Score')

In [ ]:
def plot_histogram(data, n_bins = 10):
    freq, bins = np.histogram(data, bins=n_bins)
    hist = pygal.Histogram()
    hist.add('Wide bars', list(zip(freq, bins, bins[1:])))
    return hist

In [ ]:
plot_histogram(X["score"])

In [ ]:
X['score'].min()## 29

In [ ]:
X['score'].max()## 178

In [ ]:
ls_best_raw = [x[2:] for x in ls_best]

In [ ]:
for var in ls_best:
    display(X[[var, "P_" + var]].groupby(var).last())

In [ ]:
Side_moda=X[['Side_moda', "P_" + 'Side_moda']].groupby('Side_moda').last()
Traffic_Signal=X[['Traffic_Signal', "P_" + 'Traffic_Signal']].groupby('Traffic_Signal').last()
State=X[['State', "P_" + 'State']].groupby('State').last()
Crossing=X[['Crossing', "P_" + 'Crossing']].groupby('Crossing').last()
Weekday=X[['Weekday', "P_" + 'Weekday']].groupby('Weekday').last()
Junction=X[['Junction', "P_" + 'Junction']].groupby('Junction').last()
Stop=X[['Stop', "P_" + 'Stop']].groupby('Stop').last()
Hour=X[['Hour', "P_" + 'Hour']].groupby('Hour').last()
Timezone=X[['Timezone', "P_" + 'Timezone']].groupby('Timezone').last()
Amenity=X[['Amenity', "P_" + 'Amenity']].groupby('Amenity').last()
Station=X[['Station', "P_" + 'Station']].groupby('Station').last()
Start_Lng=X[['Start_Lng', "P_" + 'Start_Lng']].groupby('Start_Lng').last()
Weather_moda=X[['Weather_moda', "P_" + 'Weather_moda']].groupby('Weather_moda').last()

In [ ]:
Z=X[ls_best].copy()
Z["Severity"]=X["Severity"].copy()

In [ ]:
Z = Z.merge(Side_moda,on=['Side_moda'],how='left')
Z = Z.merge(Traffic_Signal,on=['Traffic_Signal'],how='left')
Z = Z.merge(State,on=['State'],how='left')
Z = Z.merge(Crossing,on=['Crossing'],how='left')
Z = Z.merge(Weekday,on=['Weekday'],how='left')
Z = Z.merge(Junction,on=['Junction'],how='left')
Z = Z.merge(Stop,on=['Stop'],how='left')
Z = Z.merge(Hour,on=['Hour'],how='left')
Z = Z.merge(Timezone,on=['Timezone'],how='left')
Z = Z.merge(Amenity,on=['Amenity'],how='left')
Z = Z.merge(Station,on=['Station'],how='left')
Z = Z.merge(Start_Lng,on=['Start_Lng'],how='left')
Z = Z.merge(Weather_moda,on=['Weather_moda'],how='left')

In [ ]:
score=[x for x in Z.columns if x.startswith("P_")]

In [ ]:
Z["score"]=Z[score].sum(axis = 1)

In [ ]:
aux={}
for i in range(29,178):
        Z["ROC"]=Z["score"].map(lambda x: 1 if x < i  else 0) 
        aux[str(i)]={f"ROC":roc_auc_score(y_score=Z["ROC"], y_true=Z["Severity"])}

In [ ]:
aux## corte 71

In [ ]:
severidad=X['Severity'].value_counts(1)
severidad.iplot(kind='bar',color=['red'],theme='white')

# Tree regresor

In [1]:
import numpy as np
import pandas as pd
import pydotplus
from sklearn.tree import _tree
from IPython.display import Image 
from sklearn.externals.six import StringIO 
from sklearn.metrics import r2_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier, export_graphviz,DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',300)

In [2]:
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    features = ','.join(feature_names)
    print(f"def tree({features}):")

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print(f"{indent}if {name} <= {threshold}:")
            recurse(tree_.children_left[node], depth + 1)
            print(f"{indent}else:  # if {name} > {threshold}")
            recurse(tree_.children_right[node], depth + 1)
        else:
            print(f"{indent}return {tree_.value[node]}".format(indent, tree_.value[node]))

    recurse(0, 1)
#####################
def classification_metrics(X, y, estimator):
    ls_scores_roc = cross_val_score(estimator=estimator, X=X, y=y, scoring="roc_auc", n_jobs=-1, cv=4)
    print(f"ROC media: {np.mean(ls_scores_roc):,.2f}, desviación estándar: {np.std(ls_scores_roc)}")

In [ ]:
df=pd.read_pickle("Tabla_Proyecto_Limpia_forma1.pkl")

In [ ]:
quitar=['TMC','Start_Time', 'End_Time', 'Description', 'Street','Side','City','County','Zipcode',
          'Airport_Code','Temperature(F)', 'Humidity(%)', 'Pressure(in)'
        , 'Visibility(mi)', 'Wind_Speed(mph)',
       'Wind_Direction', 'Weather_Condition','City_Freq','Weather', 'Temperature(F)_Escalado', 'Humidity(%)_Escalado',
       'Pressure(in)_Escalado', 'Visibility(mi)_Escalado',
       'Wind_Speed(mph)_Escalado','Day']

In [ ]:
columnas=[x for x in df.columns if x not in quitar]

In [ ]:
df=df[columnas]

In [ ]:
df["Amenity"]=df["Amenity"]*1
df["Crossing"]=df["Crossing"]*1
df["Junction"]=df["Junction"]*1
df["Station"]=df["Station"]*1
df["Stop"]=df["Stop"]*1
df["Traffic_Signal"]=df["Traffic_Signal"]*1
df["Hour"]=df["Hour"].astype(str)
df["Year"]=df["Year"].astype(str)
df["Side_moda"]=(df["Side_moda"]=="R")*1
df["Weekend"]=df["Weekday"].map(lambda x: 1 if x=="Sat" or x=="Sun" else 0)

In [ ]:
df=df[df["Source"]=="MapQuest"].copy()
df=df.drop(columns=["Source"])
df=df.reset_index(drop=True)

In [ ]:
df=df[df["TimeMinutes"]<=df["TimeMinutes"].quantile(.99)].copy()
df=df.reset_index(drop=True)

In [ ]:
df1=df.sample(frac=.6,random_state=1)

In [ ]:
del df

In [ ]:
cat=["State","Timezone","Hour","Weekend",
     "Month","Side_moda","Wind_Direction_moda","Weather_moda"]#,"cut_Temperature(F)","cut_Humidity(%)","cut_Pressure(in)","cut_Visibility(mi)","cut_Wind_Speed(mph)"
cont=[x for x in df1.columns if x not in cat+["Severity","Source","Distance(mi)","Weekday",
                                            "Year","TimeMinutes"]]#+cortes

In [ ]:
aux=pd.read_csv("AUX_edos_USA.csv")
aux["poblacion"]=aux["poblacion"].replace({'601 723':601723,'1 852 994':1852994})
aux["poblacion"]=aux["poblacion"].astype(int)

In [ ]:
df1.shape

In [ ]:
df1=df1.merge(aux, left_on='State', right_on='acrónimo',how='left')
df1=df1.drop(columns=['estado', 'acrónimo'])

In [ ]:
df1.shape

In [ ]:
cont2=[x for x in df1.columns if x not in cat+["Severity","Source","Distance(mi)","Weekday",
                                            "Year","TimeMinutes"]]

In [ ]:
X = df1[cont2].join(pd.get_dummies(df1[cat]))
y = df1[["TimeMinutes"]]#Distance(mi)

In [ ]:
del df1,aux

In [ ]:
X.shape

# kbest # no los use

In [ ]:
kb=SelectKBest(score_func=f_regression, k=70)

In [ ]:
kb.fit(X,y)

In [ ]:
ls_best= [x for x, y in zip(X.columns, kb.get_support()) if y]

In [ ]:
ls_best

In [ ]:
X=X[ls_best]

# modelo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)

In [ ]:
del X,y

In [ ]:
#test=X_test.head(1)
#test.to_csv("columnas_arbol_regresivo.csv")

In [ ]:
DecisionTreeRegressor?

In [ ]:
%%time
model = DecisionTreeRegressor(max_depth=15)
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
param_dist = {"max_depth": range(10, 20),
              "max_features": ["auto","sqrt","log2"],
              "splitter": ["best","random"]}

In [ ]:
RandomizedSearchCV?

In [ ]:
%%time
model = DecisionTreeRegressor()
search = RandomizedSearchCV(param_distributions=param_dist, cv=4, n_jobs=-1, scoring="r2", estimator=model, verbose=5,n_iter=20)

In [ ]:
search.fit(X_train, y_train)

In [ ]:
search.best_score_

In [ ]:
search.best_estimator_

In [ ]:
r2_score(y_pred=search.best_estimator_.predict(X_test), y_true=y_test)

In [ ]:
#pd.to_pickle(search.best_estimator_,"arbol_regresivo.pkl")
modelo1=pd.read_pickle("arbol_regresivo.pkl")# score .29

In [ ]:
r2_score(y_pred=modelo1.predict(X_test), y_true=y_test)

In [ ]:
list(zip(X_test.columns,modelo1.feature_importances_))

In [ ]:
dot_data = StringIO()
export_graphviz(modelo1, out_file=dot_data, feature_names=X_train.columns,
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
tree_to_code(modelo1,feature_names=X_train.columns)

# Tree Classifier

In [3]:
df=pd.read_pickle("Tabla_Proyecto_Limpia_forma1.pkl")

In [4]:
quitar=['TMC','Start_Time', 'End_Time', 'Description', 'Street','Side','City','County','Zipcode',
          'Airport_Code','Temperature(F)', 'Humidity(%)', 'Pressure(in)'
        , 'Visibility(mi)', 'Wind_Speed(mph)',
       'Wind_Direction', 'Weather_Condition','City_Freq','Weather', 'Temperature(F)_Escalado', 'Humidity(%)_Escalado',
       'Pressure(in)_Escalado', 'Visibility(mi)_Escalado',
       'Wind_Speed(mph)_Escalado','Day']

In [5]:
columnas=[x for x in df.columns if x not in quitar]

In [6]:
df=df[columnas]

In [7]:
df["Amenity"]=df["Amenity"]*1
df["Crossing"]=df["Crossing"]*1
df["Junction"]=df["Junction"]*1
df["Station"]=df["Station"]*1
df["Stop"]=df["Stop"]*1
df["Traffic_Signal"]=df["Traffic_Signal"]*1
df["Hour"]=df["Hour"].astype(str)
df["Year"]=df["Year"].astype(str)
df["Side_moda"]=(df["Side_moda"]=="R")*1
df["Weekend"]=df["Weekday"].map(lambda x: 1 if x=="Sat" or x=="Sun" else 0)

In [8]:
df=df[df["Source"]=="MapQuest"].copy()
df=df.drop(columns=["Source"])
df=df.reset_index(drop=True)

In [9]:
df=df[df["TimeMinutes"]<=df["TimeMinutes"].quantile(.99)].copy()
df=df.reset_index(drop=True)

In [10]:
dic={1:"Poca severo",2:"Poca severo",3:"Muy severo",4:"Muy severo"}
df["target"]=df["Severity"].replace(dic)

In [11]:
cat=["State","Timezone","Hour","Weekend",
     "Month","Side_moda","Wind_Direction_moda","Weather_moda"]
cont=[x for x in df.columns if x not in cat+["Severity","Source","Distance(mi)","Weekday",
                                            "Year","TimeMinutes","target"]]

In [12]:
X = df[cont].join(pd.get_dummies(df[cat]))
y = df[["target"]]#target,Severity

In [13]:
del df

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)

In [15]:
del X,y

In [17]:
#test=X_test.head(1)
#test.to_csv("columnas_arbol_clasificación.csv")

# Modelo

In [ ]:
DecisionTreeClassifier?

In [ ]:
%%time
model=DecisionTreeClassifier(max_depth=15,class_weight="balanced")
model.fit(X_train,y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
confusion_matrix(y_pred=model.predict(X_test),y_true=y_test)

In [ ]:
classification_metrics(X=X_train, y=y_train, estimator=model)

In [ ]:
param_dist = {"max_depth": range(10, 20),
              "max_features": ["auto", "sqrt", "log2"],
              "min_samples_leaf": [x for x in range(1000,10000, 100)], 
              "criterion": ["gini", "entropy"],
              "splitter": ["best", "random"],
              "class_weight": ["balanced"]
             }

In [ ]:
modelo=DecisionTreeClassifier()
search = RandomizedSearchCV(param_distributions=param_dist, cv=4, n_jobs=-1, scoring="roc_auc", 
                            estimator=modelo, verbose=5,n_iter=50)

In [ ]:
search.fit(X_train, y_train)

In [ ]:
search.best_estimator_.score(X_test, y_test)

In [ ]:
search.best_estimator_.score(X_train, y_train)

In [ ]:
#pd.to_pickle(model,"arbolclasificar.pkl") roc.772

# XGBoost

#### usa la tabla del tree classifier

In [ ]:
from xgboost.sklearn import XGBClassifier

In [ ]:
XGBClassifier?

In [ ]:
%%time
model=XGBClassifier()
model.fit(X_train,y_train.values.ravel())

In [ ]:
model.score(X_train, y_train.values.ravel())

In [ ]:
model.score(X_test, y_test.values.ravel())

In [ ]:
y_true=(y_test=='Muy severo')*1
y_pred=(model.predict(X_test)=='Muy severo')*1

In [ ]:
#pd.to_pickle(model,"xgbclassifier_proyecto.pkl") ###0.8446545466135966 

In [ ]:
roc_auc_score(y_pred,y_true)

In [ ]:
def classification_metrics(X, y, estimator):
    ls_scores_roc = cross_val_score(estimator=estimator, X=X, y=y, scoring="roc_auc", n_jobs=-1, cv=4)
    print(f"ROC media: {np.mean(ls_scores_roc):,.2f}, desviación estándar: {np.std(ls_scores_roc)}")

In [ ]:
#classification_metrics(X=X_train, y=y_train.values.ravel(), estimator=model)